### Calibration interrogation
This notebook examines the parameter outputs from the calibration algorithm.

In [ ]:
import numpy as np
import arviz as az

from autumn.infrastructure.remote import springboard
from aust_covid.inputs import get_ifrs
from aust_covid.utils import add_image_to_doc
from aust_covid.calibration import get_priors
from emutools.tex import DummyTexDoc, StandardTexDoc
from emutools.parameters import load_param_info
from emutools.calibration import plot_param_progression, plot_posterior_comparison, tabulate_param_results
from inputs.constants import PROJECT_PATH, SUPPLEMENT_PATH

In [ ]:
dummy_doc = DummyTexDoc()
param_info = load_param_info()
parameters = param_info['value'].to_dict() | get_ifrs(dummy_doc)
priors = get_priors()
prior_names = [p.name for p in priors]

In [ ]:
rts = springboard.task.RemoteTaskStore()
rts.cd('projects/aust_covid/opt_experiments')
mt = rts.get_managed_task('2023-09-18T1351-lhs_constrained0.67_opt100_10k10k_demz')
mt.download_all()

In [ ]:
idata = az.from_netcdf(mt.local.path / 'output/calibration_out.nc')

In [ ]:
# Burn
idata = idata.sel(draw=np.s_[1000:])

### Parameter traces with estimated posterior densities by chain

In [ ]:
n_half_priors = round(len(priors) / 2)
first_priors = prior_names[:n_half_priors]
last_priors = prior_names[n_half_priors:]
trace_fig0 = plot_param_progression(idata, param_info, request_vars=first_priors)

In [ ]:
trace_fig1 = plot_param_progression(idata, param_info, request_vars=last_priors)

### Comparison of parameter posteriors to priors

In [ ]:
comp_fig0 = plot_posterior_comparison(idata, priors, first_priors, param_info['abbreviations'].to_dict(), 0.995)

In [ ]:
comp_fig1 = plot_posterior_comparison(idata, priors, last_priors, param_info['abbreviations'].to_dict(), 0.995)

### Write to TeX document

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'calibration', 'Calibration', 'austcovid')
app_doc.include_table(tabulate_param_results(idata, priors, param_info), section='Calibration', col_splits=[0.142] * 7, table_width=12.0, longtable=True)
trace_figs = [trace_fig0, trace_fig1]
comp_figs = [comp_fig0, comp_fig1]
for i_fig in range(2):
    add_image_to_doc(trace_figs[i_fig], f'trace_fig_{i_fig}', f'Parameter posteriors and traces by chain, {i_fig}.', app_doc, 'Calibration')
    add_image_to_doc(comp_figs[i_fig], f'comp_fig_{i_fig}', f'Comparison of posterior densities against prior distributions, {i_fig}.', app_doc, 'Calibration')
app_doc.write_doc()